# Convert Atlas dataset into Oceantide format

In [ ]:
import itertools
import warnings
from pathlib import Path
import xarray as xr

from oceantide import read_otis_atlas_netcdf

warnings.filterwarnings("ignore", category=RuntimeWarning) 


In [ ]:
def chunked_iterable(iterable, size):
    """Iterate through array over chunks with specific size."""
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, size))
        if not chunk:
            break
        yield chunk

## Convert by iterating through longitudes over chunks of size 200 to handle memory
The Atlas dataset is very large. It is composed by one file per constituent for elevation and transports at 1/30 degree resolution. Reading in the full dataset into oceantide format cannot be handled unless available RAM is very large. Here we overcome this limitation by converting and concatenating longitude chunks at a time. 

In [ ]:
atlas_path = Path("/data/tide/tpxo9v4_atlas/TPXO9_atlas_nc")

lons = xr.open_dataset(atlas_path / "grid_tpxo9_atlas_v4.nc").lon_z.values

chunksizes = 100
itersize = 200

for ind in chunked_iterable(range(lons.size), size=itersize):
    x0 = lons[ind[0]]
    x1 = lons[ind[-1]]
    print(f"Writing lon chunk {ind[0]}")
    dset = read_otis_atlas_netcdf(atlas_path, x0=x0, x1=x1, nxchunk=itersize, nychunk=None)
    dset = dset.chunk({"con": None, "lon": chunksizes, "lat": chunksizes})
    if ind[0] == 0:
        mode = "w"
        append_dim = None
    else:
        mode = "a"
        append_dim = "lon"
    dset.to_zarr("./atlas.zarr", mode=mode, consolidated=True, append_dim=append_dim)
